In [27]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from prophet.serialize import model_to_json

In [8]:
df=pd.read_csv("exog_variables (1).csv")

In [9]:
df_cpi = df[["DATE","CPI"]]

In [10]:
df_cpi['DATE'] = pd.to_datetime(df_cpi['DATE'])

df_cpi.set_index('DATE', inplace=True)

In [11]:
df_cpi = df_cpi.resample('MS').mean()

In [12]:
df_cpi=df_cpi.reset_index()

In [13]:
df_cpi_latest = df_cpi.tail(32)

In [14]:
df_cpi_latest['ds'] = pd.to_datetime(df_cpi_latest['DATE'])
df_cpi_latest['y'] = df_cpi_latest["CPI"]

In [15]:
train_data = df_cpi_latest.head(18)
test_data = df_cpi_latest.tail(5)

In [16]:
df_log_cpi = pd.DataFrame(columns=["Yearly_order","Train MAPE","Test MAPE"])
    
yearly_order=[2,3,4,5,6,7,8,9,10,15,17,20,25,26,27,28,29,30]
    

for year in yearly_order:
    model=Prophet(seasonality_mode='additive',growth="linear",weekly_seasonality=False, yearly_seasonality=False,daily_seasonality=False)

    model.add_seasonality(name='yearly',period=365.25,fourier_order=year)

    model.fit(train_data)
    future = model.make_future_dataframe(periods=len(test_data), freq='MS')


    train_mape=mean_absolute_percentage_error(train_data['y'],model.predict(train_data).yhat)*100

    forecast = model.predict(future)
    forecast_test = forecast[-len(test_data):]
    y_true = test_data['y'].values
    y_pred = forecast_test['yhat']
    mape = mean_absolute_percentage_error(y_true,y_pred)
    print(year,train_mape,mape*100)
    df_log_cpi = df_log_cpi.append({'Yearly_order':year,"Train MAPE":train_mape,"Test MAPE":mape*100},ignore_index = True)
        

20:26:18 - cmdstanpy - INFO - Chain [1] start processing
20:26:18 - cmdstanpy - INFO - Chain [1] done processing
20:26:18 - cmdstanpy - INFO - Chain [1] start processing


2 0.08149105778118285 1.85871659908756


20:26:21 - cmdstanpy - INFO - Chain [1] done processing
20:26:21 - cmdstanpy - INFO - Chain [1] start processing


3 1.5364007062071008e-05 5.0382902525220485


20:26:26 - cmdstanpy - INFO - Chain [1] done processing
20:26:26 - cmdstanpy - INFO - Chain [1] start processing


...

28 0.0020673257271474994 100.002538744896


20:27:59 - cmdstanpy - INFO - Chain [1] done processing
20:27:59 - cmdstanpy - INFO - Chain [1] start processing


29 0.0027094154567177758 93.57724559524803


20:28:01 - cmdstanpy - INFO - Chain [1] done processing


30 0.0016524921917501674 121.74055217585284


In [17]:
df_log_cpi=df_log_cpi.sort_values("Test MAPE")
df_log_cpi = df_log_cpi.reset_index(drop=True)
df_log_cpi

,Yearly_order,Train MAPE,Test MAPE
0,7.0,0.000051,0.948144
1,2.0,0.081491,1.858717
2,6.0,0.000047,2.227119
3,9.0,0.000013,2.641413
4,4.0,0.000050,3.409789
5,5.0,0.000034,4.326346
6,3.0,0.000015,5.038290
7,17.0,0.000294,8.417438
8,8.0,0.000335,17.639827
9,26.0,0.000328,21.292700


In [19]:
yearly_order_value = df_log_cpi.at[0, 'Yearly_order']

In [23]:
model=Prophet(growth="linear",weekly_seasonality=False, yearly_seasonality=False,daily_seasonality=False)
model.add_seasonality(name='yearly',period=365.25,fourier_order=int(yearly_order_value))
model.fit(train_data)

20:31:20 - cmdstanpy - INFO - Chain [1] start processing
20:31:26 - cmdstanpy - INFO - Chain [1] done processing


In [28]:
current_date = datetime.now().strftime('%d%m%Y')

file_name = f'{current_date}.json'

with open(file_name, 'w') as fout:
    fout.write(model_to_json(model))